# Credit Card Fraud Detection::

Download dataset from this link:

https://www.kaggle.com/mlg-ulb/creditcardfraud

# Description about dataset::

The datasets contains transactions made by credit cards in September 2013 by european cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. 


### Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

# WORKFLOW :

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Standardized the Input Variables. 

4.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

5.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

6.Compilation Step (Note : Its a Binary problem , select loss , metrics according to it)

7.Train the Model with Epochs (100).

8.If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .

9.Prediction should be > 92%
10.Evaluation Step
11Prediction


# Task::

## Identify fraudulent credit card transactions.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns  ###
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.optimizers import Adam, SGD

from sklearn.model_selection import train_test_split

data1 = pd.read_csv("creditcard.csv")
credit = data1.copy()
# df =  data1.copy

In [2]:
credit.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
# credit.isnull().sum()
credit.isnull().any()

Time      False
V1        False
V2        False
V3        False
V4        False
V5        False
V6        False
V7        False
V8        False
V9        False
V10       False
V11       False
V12       False
V13       False
V14       False
V15       False
V16       False
V17       False
V18       False
V19       False
V20       False
V21       False
V22       False
V23       False
V24       False
V25       False
V26       False
V27       False
V28       False
Amount    False
Class     False
dtype: bool

In [4]:
credit.dtypes

Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object

In [5]:
credit.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [6]:
credit.V1.value_counts()

 2.055797    77
 1.245674    77
 2.053311    62
 1.302378    60
 2.040211    53
             ..
-0.069327     1
-1.619420     1
 1.174923     1
 2.157306     1
 2.287784     1
Name: V1, Length: 275663, dtype: int64

In [7]:
# credit.info()
np.where(credit.applymap(lambda x: x == ''))
# np.where(credit.Amount.apply(lambda x: x == 0)) ## finding the outliers

(array([], dtype=int64), array([], dtype=int64))

In [49]:
Not_fraud = credit[credit["Class"]==0] #output_variable  ## 284,807 Not Fraud Data
fraud = credit[credit["Class"]==1]  #output_Variable     ### 492 is Fraud data
# Not_fraud.shape,fraud.shape

###       retrive random sample according to len()
Not_fraud = Not_fraud.sample(fraud.shape[0])   ### 492 * 2 = 984 sample SHAPE is 984,31
data = fraud.append(Not_fraud,ignore_index=True) ## you can use pd.concate or append both then skip index
# data["Class"].value_counts()  shows this ----   0 >> 984    1>>492

x_data = data.drop(columns="Class", axis=0) ## drop the output_varibale from x_data
y_labels = data["Class"] ### output_variable targets_var Or Dependent varaible y_labels 

In [50]:
print('Not_fraud',Not_fraud.shape)
print('fraud',fraud.shape)
print()
print('x_data',x_data.shape)
print('y_labels',y_labels.shape)

Not_fraud (492, 31)
fraud (492, 31)

x_data (984, 30)
y_labels (984,)


In [51]:
###                                                     radom_state stop random selection
###  test_size=0.3this for 70/30 ratio                          ## random state is seed
x_train, x_test, y_train, y_test = train_test_split(x_data,y_labels, test_size=0.3,random_state=0, stratify = y_labels)

In [52]:
print(x_train.shape)
print(x_test.shape)
print()
print(y_train.shape)
print(y_test.shape)

(688, 30)
(296, 30)

(688,)
(296,)


In [53]:
########## Normalization of independent
mean = x_train.mean(axis=0) # taking the mean of 
x_train -= mean
std = x_train.std(axis=0)
x_train /= std

x_test = (x_test - mean) /std

# # Convert into Numpy Array
# x_train = np.array(x_train)
# x_test = np.array(x_test)
# Convert into dtypes

y_train= np.array(y_train.astype('float32'))
y_test= np.array(y_test.astype('float32'))

In [60]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10,activation='relu',input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),    
    tf.keras.layers.Dense(8,activation='relu'),
    tf.keras.layers.Dropout(0.5),     
    tf.keras.layers.Dense(6,activation='relu'),     
    tf.keras.layers.Dropout(0.5),     
    tf.keras.layers.Dense(1, activation='sigmoid')])#
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [61]:
# verbose=0
history = model.fit(x_train, y_train,epochs=100,validation_split=.2)
# validation_batch_size=150, validation_steps=10)

Epoch 1/100
18/18 [==============================] - 1s 8ms/step - loss: 0.8809 - accuracy: 0.4674 - val_loss: 0.6731 - val_accuracy: 0.4855
Epoch 2/100
18/18 [==============================] - 0s 2ms/step - loss: 0.8116 - accuracy: 0.5252 - val_loss: 0.6545 - val_accuracy: 0.5652
Epoch 3/100
18/18 [==============================] - 0s 3ms/step - loss: 0.7607 - accuracy: 0.5491 - val_loss: 0.6419 - val_accuracy: 0.5942
Epoch 4/100
18/18 [==============================] - 0s 3ms/step - loss: 0.8120 - accuracy: 0.5218 - val_loss: 0.6248 - val_accuracy: 0.6377
Epoch 5/100
18/18 [==============================] - 0s 2ms/step - loss: 0.8038 - accuracy: 0.5648 - val_loss: 0.6137 - val_accuracy: 0.6957
Epoch 6/100
18/18 [==============================] - 0s 3ms/step - loss: 0.7042 - accuracy: 0.5831 - val_loss: 0.6037 - val_accuracy: 0.7536
Epoch 7/100
18/18 [==============================] - 0s 3ms/step - loss: 0.6424 - accuracy: 0.6412 - val_loss: 0.5922 - val_accuracy: 0.7754
Epoch 8/100
1

Epoch 59/100
18/18 [==============================] - 0s 3ms/step - loss: 0.4139 - accuracy: 0.8164 - val_loss: 0.4505 - val_accuracy: 0.8551
Epoch 60/100
18/18 [==============================] - 0s 3ms/step - loss: 0.4285 - accuracy: 0.7855 - val_loss: 0.4527 - val_accuracy: 0.8551
Epoch 61/100
18/18 [==============================] - 0s 3ms/step - loss: 0.3964 - accuracy: 0.8142 - val_loss: 0.4564 - val_accuracy: 0.8551
Epoch 62/100
18/18 [==============================] - 0s 3ms/step - loss: 0.3978 - accuracy: 0.8217 - val_loss: 0.4517 - val_accuracy: 0.8551
Epoch 63/100
18/18 [==============================] - 0s 3ms/step - loss: 0.3967 - accuracy: 0.8304 - val_loss: 0.4528 - val_accuracy: 0.8551
Epoch 64/100
18/18 [==============================] - 0s 3ms/step - loss: 0.3709 - accuracy: 0.8443 - val_loss: 0.4544 - val_accuracy: 0.8623
Epoch 65/100
18/18 [==============================] - 0s 3ms/step - loss: 0.3519 - accuracy: 0.8403 - val_loss: 0.4538 - val_accuracy: 0.8623
Epoch 

In [62]:
test_loss, test_acc = model.evaluate(x_test,y_test)

10/10 [==============================] - 0s 1ms/step - loss: 0.3542 - accuracy: 0.9054


In [56]:
test_acc * 100

93.00225973129272

In [ ]:
# model.summary()
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history

loss = history_dict['loss']
acc = history_dict['val_loss']

epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'red', label='loss')
plt.plot(epochs, acc, 'bo', label='accuracy')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history

loss = history_dict['accuracy']
acc = history_dict['val_accuracy']

epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'red', label='loss')
plt.plot(epochs, acc, 'bo', label='accuracy')
plt.legend()
plt.show()

In [63]:
# predicting the test set results
y_pred = model.predict(x_test).astype(dtype="u8")

In [64]:
y_pred[:11]

array([[0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]], dtype=uint64)

In [65]:
y_test[:9]

array([0., 0., 1., 0., 1., 0., 1., 0., 0.], dtype=float32)

In [ ]:
# import matplotlib.pyplot as plt

# epochs = range(1, 9 + 1)
# plt.plot(epochs, y_pred[:9], 'red', label='y_pred')
# plt.plot(epochs, y_test[:9], 'bo', label='y_test')
# plt.legend()
# plt.show()

In [ ]:
x_train.shape

In [ ]:
credit.shape

In [ ]:
len(train)/len(credit)  ### split 70 % data

In [ ]:
len(test)/len(credit)  ### split 30 % data

In [ ]:
0.2 * len(credit)

In [ ]:
0.2 * len(credit)/ len(train)